## Set up

In [1]:
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## Wikidata queries

### 1. Retrieve "films based on literary works"

In [6]:
fblw_query ="""
SELECT (COUNT(DISTINCT ?film) AS ?film)
WHERE {

    ?film wdt:P31/wdt:P279* wd:Q11424 .
    ?novel wdt:P31/wdt:P279* wd:Q7725634 .
    # ?novel wdt:P7937 wd:Q8261 .
    ?film wdt:P144 ?novel . 

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
films_based_on_literary_works = query_wikidata(endpoint, fblw_query, user_agent, True)
films_based_on_literary_works

,film
0,9492


### 2. Retrieve "films based on novels"

In [7]:
fbn_query = """
SELECT (COUNT(DISTINCT ?film) AS ?film)
WHERE {

    ?film wdt:P31/wdt:P279* wd:Q11424 .
    #?novel wdt:P31/wdt:P279* wd:Q7725634 .
    ?novel wdt:P7937 wd:Q8261 .
    ?film wdt:P144 ?novel . 

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
films_based_on_novels = query_wikidata(endpoint, fbn_query, user_agent, True)
films_based_on_novels

,film
0,4957


### 3. Retrieve "films based on novels" excluding the ones that cannot be fount on the English Wikipedia

In [8]:
fbn_enwiki_query = """
SELECT (COUNT(DISTINCT ?film) AS ?film)
WHERE {

    ?film wdt:P31/wdt:P279* wd:Q11424 .
    ?novel wdt:P7937 wd:Q8261 .
    ?film wdt:P144 ?novel .
    
    ?wikipage schema:about ?film .
    ?wikipage schema:isPartOf <https://en.wikipedia.org/>;
                
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
films_based_on_novels_enwiki = query_wikidata(endpoint, fbn_enwiki_query, user_agent, True)
films_based_on_novels_enwiki

,film
0,3986
